You can use this notebook to visulalize the quantum correlators. You need to perform a semiclassical simulation (with or without backreaction) and store "data", and "correlators". It assumes that both data-types are saved at the same timestpes. If not, you need to modify appropriately.

In [ ]:
# load packages
using HDF5
using LaTeXStrings
using Plots ; pythonplot()
using DelimitedFiles
using SpecialFunctions
using FunctionZeros
using Interpolations
using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
base = "../examples/quantum_runs/"

#par1 = "a0.0_b0.0_c1.0_rmax30.0_tmax8.0_cfl0.0625_sigma0.02_overMp2_1.0_reg_true_backreact_false_mPV1.0_dk_denom_30_kmax20.0_lmax60.0"
par2 = "a1.25_b0.0_c1.0_rmax30.0_tmax8.0_cfl0.0625_sigma0.02_overMp2_1.0_reg_true_backreact_false_mPV1.0_dk_denom_30_kmax30.0_lmax90.0"
par3 = "a0.75_b0.0_c1.0_rmax30.0_tmax8.0_cfl0.0625_sigma0.02_overMp2_1.0_reg_true_backreact_false_mPV1.0_dk_denom_30_kmax30.0_lmax90.0"

#dir1 = base*par1
dir2 = base*par2
dir3 = base*par3

In [ ]:
# set the number of grid points you used
D = 3
Nr = 128*2^D + 3

# make directory to save figures
out_dir = "./correlators_tr/a0.75_a1.25_comparison_kmax30_lmax90/resol_D$(D)"
if ispath(out_dir)==false
    mkpath(out_dir)
end

# load the r grid
r = h5read(dir2*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]

# list all available iterations (and corresponding files)
#(its1, all_filenames)           = list_h5_files(dir1*"/data_$(Nr)", prefix="data_");
#(its_crlt1, all_filenames_crlt) = list_h5_files(dir1*"/data_$(Nr)", prefix="correlators_");

(its2, all_filenames)           = list_h5_files(dir2*"/data_$(Nr)", prefix="data_");
(its_crlt2, all_filenames_crlt) = list_h5_files(dir2*"/data_$(Nr)", prefix="correlators_");

(its3, all_filenames)           = list_h5_files(dir3*"/data_$(Nr)", prefix="data_");
(its_crlt3, all_filenames_crlt) = list_h5_files(dir3*"/data_$(Nr)", prefix="correlators_");

@assert its2 ≈ its3
@assert its_crlt2 ≈ its_crlt3

# classical data
println("length(its2) = ",length(its2))
println("its = ",its2)

# correlators
println("length(its_crlt2) = ",length(its_crlt2))
println("its_crlt = ",its_crlt2)

Define a function to plot the real part of equal time correlators. The structure of the correlators is given before the function. You can modify it to plot the imaginary part

In [ ]:
"""
for the correlators the format (r, r)
    it is the 2pt function for the fields:
    1) ∂t_utld_kl = r^l*α*πq/(B*A^0.5)
    2) ∂r_utld_kl = l*r^(l-1)*uq + r^l*ψq
    3) utld_kl = uq*r^l
    4) πq
    5) ψq
"""

function calc_crlt_AH(dir, it, crlt_i)

    (its, all_filenames)           = list_h5_files(dir*"/data_$(Nr)", prefix="data_");
    (its_crlt, all_filenames_crlt) = list_h5_files(dir*"/data_$(Nr)", prefix="correlators_");
    
    # if you have not saved data and correlators on the same timesteps
    # you need to change it below, accordingly
    it_data = its[it]
    it_str_data  = lpad(it_data, 4, "0")
    
    it_crlt = its_crlt[it]
    it_str_crlt  = lpad(it_crlt, 4, "0")
    
    v = h5read(dir*"/data_$(Nr)/data_$(it_str_data).h5","v")
    A  = v[:,4]
    B  = v[:,5]
    KB = v[:,9]
    
    crlt = h5read(dir*"/data_$(Nr)/correlators_$(it_str_crlt).h5","correlators")
    
    # if there is an error when loading the time attribute, try to change "./" to "." or to "/" 
    t_data   =  h5readattr(dir*"/data_$(Nr)/data_$(it_str_data).h5", "/")["time"]
    t_crlt   =  h5readattr(dir*"/data_$(Nr)/correlators_$(it_str_crlt).h5", "/")["time"]
    AH = find_AH(r, B, A, KB)

    # checks if time instant is the same for all types of data
    @assert t_data ≈ t_crlt

    return AH, real(crlt[:,:,crlt_i]), t_data
end


In [ ]:
# index of min and max, in r domain
ri_min = 3
ri_max = 550#Nr
# timestep (counts within its_crlt list
ti = 35#69
# which correlator you want (see previous cell for the structure). There are 5 different (1-5).
crlt_i =3 

AH2, crlt2, t = calc_crlt_AH(dir2, ti, crlt_i)
AH3, crlt3, t = calc_crlt_AH(dir3, ti, crlt_i)
# color
#clr = :acton
#clr = :devon
clr = :lipari

contourf(r[ri_min:ri_max], r[ri_min:ri_max], crlt2[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
xaxis="r", 
yaxis="r",
title = L"C(r,r)",
colorbar=false,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10
)

plot!([AH2], seriestype="hline", label="t = $(t)", linewidth=3, color = "white")
plot!([AH2], seriestype="vline", label="AH", linewidth=3, color = "black")
p2 = plot!([AH2], seriestype="hline", label="", linewidth=3, color = "black")

p3= contourf(r[ri_min:ri_max], r[ri_min:ri_max], crlt3[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
xaxis="r", 
yaxis="",
yticks=([0,5,10,15],["","","",""]),
title = L"C(r,r)",
clim=( min(minimum(crlt2[ri_min:ri_max]),minimum(crlt3[ri_min:ri_max])), max(maximum(crlt2[ri_min:ri_max]),maximum(crlt3[ri_min:ri_max]))),
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10
)

#plot!([AH3], seriestype="vline", label="AH", linewidth=3, color = "black")
#p3 = plot!([AH3], seriestype="hline", label="AH", linewidth=3, color = "black")

plt = plot(
    p2, p3,
    layout = grid(1, 2, spacing=(0.0, 0.0)), size = (1200, 400)
)

# uncomment below if you want to save fig
#savefig(plt, out_dir*"/crlt_$(crlt_i)-$(ti).png")

In [ ]:
function load_crtl23(ti, crlt_i)
    it_data = its3[it]
    it_str_data  = lpad(it_data, 4, "0")
    it_crlt = its_crlt3[it]
    it_str_crlt  = lpad(it_crlt, 4, "0")
    v = h5read(dir2*"/data_$(Nr)/data_$(it_str_data).h5","v")
    A  = v[:,4]
    B  = v[:,5]
    KB = v[:,9]
    AH = find_AH(r, B, A, KB)  
    crlt3 = h5read(dir3*"/data_$(Nr)/correlators_$(it_str_crlt).h5","correlators")
    crlt2 = h5read(dir2*"/data_$(Nr)/correlators_$(it_str_crlt).h5","correlators")
    t   =  h5readattr(dir3*"/data_$(Nr)/data_$(it_str_data).h5", "/")["time"]
    return real(crlt2[:,:,crlt_i]), real(crlt3[:,:,crlt_i]), t, AH
end

In [ ]:
ri_min = 3
ri_max = 345
ri = 75 # r slice
# heatmap color
clr = :lipari
# slice color
clr_s = "grey"
# apparent horizon color
clr_ah = "black"
#which correlator
crlt_i = 3
# legend and label size
ls = 10

##########################################################

it = 1
crlt2, crlt3, t, AH = load_crtl23(it, crlt_i)
clim=( 
    min(minimum(crlt2[ri_min:ri_max,ri_min:ri_max]),minimum(crlt3[ri_min:ri_max,ri_min:ri_max])), 
    max(maximum(crlt2[ri_min:ri_max,ri_min:ri_max]),maximum(crlt3[ri_min:ri_max,ri_min:ri_max])))

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt3[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
        title = "subcritical",
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
colorbar=false,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="r",
yticks=([0,2.5,5,7.5,10],["0","2.5","5","7.5","10"]),
)

#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
p11 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt2[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
        title = "supercritical",
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="",
yticks=([0,2.5,5,7.5,10],["","","","",""]),
)

plot!([r[ri]], seriestype="hline", label="t=$(t)", linewidth=1.5, color = "white", legendfontsize=ls, style=:dash)
#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
p12 = plot!([r[ri]], seriestype="hline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

p13 = plot(r[ri_min:ri_max],  crlt2[ri_min:ri_max, ri], label="supercritical", frame = true, linewidth=2)
p13 = plot!(r[ri_min:ri_max],  crlt3[ri_min:ri_max, ri], label="subcritical", linewidth=2, style=:dash,
         xlim = (0,r[ri_max]),
    #yticks=([0.02,0.04,0.06,0.08],["0.02","0.04","0.06","0.08"]),
    xticks=([0,2.5,5,7.5,10],["","","","",""])

)
p13 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

##########################################################

it = 7
crlt2, crlt3, t, AH = load_crtl23(it, crlt_i)
clim=( 
    min(minimum(crlt2[ri_min:ri_max,ri_min:ri_max]),minimum(crlt3[ri_min:ri_max,ri_min:ri_max])), 
    max(maximum(crlt2[ri_min:ri_max,ri_min:ri_max]),maximum(crlt3[ri_min:ri_max,ri_min:ri_max])))

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt3[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
colorbar=false,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="r",
yticks=([0,2.5,5,7.5,10],["0","2.5","5","7.5",""]),
)

#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
p21 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt2[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="",
yticks=([0,2.5,5,7.5,10],["","","","",""]),)

plot!([r[ri]], seriestype="hline", label="t=$(t)", linewidth=1.5, color = "white", legendfontsize=ls, style=:dash)
#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
p22 = plot!([r[ri]], seriestype="hline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

plot(r[ri_min:ri_max],  crlt2[ri_min:ri_max, ri], label="supercritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  crlt3[ri_min:ri_max, ri], label="subcritical", linewidth=2, style=:dash,
         xlim = (0,r[ri_max]), 
    #yticks=([0.02,0.04,0.06,0.08],["0.02","0.04","0.06","0.08"]),
    xticks=([0,2.5,5,7.5,10],["","","","",""])

)
p23 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

##########################################################

it = 11
crlt2, crlt3, t, AH = load_crtl23(it, crlt_i)
clim=( 
    min(minimum(crlt2[ri_min:ri_max,ri_min:ri_max]),minimum(crlt3[ri_min:ri_max,ri_min:ri_max])), 
    max(maximum(crlt2[ri_min:ri_max,ri_min:ri_max]),maximum(crlt3[ri_min:ri_max,ri_min:ri_max])))

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt3[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
colorbar=false,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="r",
yticks=([0,2.5,5,7.5,10],["0","2.5","5","7.5",""]),
)

#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
p31 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt2[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="",
yticks=([0,2.5,5,7.5,10],["","","","",""]),
)

plot!([r[ri]], seriestype="hline", label="t=$(t)", linewidth=1.5, color = "white", legendfontsize=ls, style=:dash)
#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
plot!([r[ri]], seriestype="hline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)
#plot!([AH], seriestype="hline", label="", linewidth=1.5, color = clr_ah, legendfontsize=ls)
p32 = plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = clr_ah, legendfontsize=ls)

plot(r[ri_min:ri_max],  crlt2[ri_min:ri_max, ri], label="supercritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  crlt3[ri_min:ri_max, ri], label="subcritical", linewidth=2, style=:dash,
         xlim = (0,r[ri_max]), 
    #yticks=([0.02,0.04,0.06,0.08],["0.02","0.04","0.06","0.08"]),
    xticks=([0,2.5,5,7.5,10],["","","","",""])

)
plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = clr_ah, legendfontsize=ls)
p33 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

##########################################################

it = 20
crlt2, crlt3, t, AH = load_crtl23(it, crlt_i)
clim=( 
    min(minimum(crlt2[ri_min:ri_max,ri_min:ri_max]),minimum(crlt3[ri_min:ri_max,ri_min:ri_max])), 
    max(maximum(crlt2[ri_min:ri_max,ri_min:ri_max]),maximum(crlt3[ri_min:ri_max,ri_min:ri_max])))

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt3[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
colorbar=false,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="r",
yticks=([0,2.5,5,7.5,10],["0","2.5","5","7.5",""]),
)

#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
p41 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt2[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="",
yticks=([0,2.5,5,7.5,10],["","","","",""]),
)

plot!([r[ri]], seriestype="hline", label="t=$(t)", linewidth=1.5, color = "white", legendfontsize=ls, style=:dash)
#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
plot!([r[ri]], seriestype="hline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)
#plot!([AH], seriestype="hline", label="", linewidth=1.5, color = clr_ah, legendfontsize=ls)
p42 = plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = clr_ah, legendfontsize=ls)

plot(r[ri_min:ri_max],  crlt2[ri_min:ri_max, ri], label="supercritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  crlt3[ri_min:ri_max, ri], label="subcritical", linewidth=2, style=:dash,
         xlim = (0,r[ri_max]), 
    #yticks=([0.02,0.04,0.06,0.08],["0.02","0.04","0.06","0.08"]),
    xticks=([0,2.5,5,7.5,10],["","","","",""])

)
plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = clr_ah, legendfontsize=ls)
p43 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

##########################################################

it = 38
crlt2, crlt3, t, AH = load_crtl23(it, crlt_i)
clim=( 
    min(minimum(crlt2[ri_min:ri_max,ri_min:ri_max]),minimum(crlt3[ri_min:ri_max,ri_min:ri_max])), 
    max(maximum(crlt2[ri_min:ri_max,ri_min:ri_max]),maximum(crlt3[ri_min:ri_max,ri_min:ri_max])))

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt3[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
colorbar=false,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="r",
yticks=([0,2.5,5,7.5,10],["0","2.5","5","7.5",""]),
)

#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
p51 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt2[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="",
yticks=([0,2.5,5,7.5,10],["","","","",""]),
)

plot!([r[ri]], seriestype="hline", label="t=$(t)", linewidth=1.5, color = "white", legendfontsize=ls, style=:dash)
#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
plot!([r[ri]], seriestype="hline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)
#plot!([AH], seriestype="hline", label="", linewidth=1.5, color = clr_ah, legendfontsize=ls)
p52 = plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = clr_ah, legendfontsize=ls)

plot(r[ri_min:ri_max],  crlt2[ri_min:ri_max, ri], label="supercritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  crlt3[ri_min:ri_max, ri], label="subcritical", linewidth=2, style=:dash,
         xlim = (0,r[ri_max]), 
    #yticks=([0.02,0.04,0.06,0.08],["0.02","0.04","0.06","0.08"]),
    xticks=([0,2.5,5,7.5,10],["","","","",""])

)
plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = clr_ah, legendfontsize=ls)
p53 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

##########################################################

it = 50
crlt2, crlt3, t, AH = load_crtl23(it, crlt_i)
clim=( 
    min(minimum(crlt2[ri_min:ri_max,ri_min:ri_max]),minimum(crlt3[ri_min:ri_max,ri_min:ri_max])), 
    max(maximum(crlt2[ri_min:ri_max,ri_min:ri_max]),maximum(crlt3[ri_min:ri_max,ri_min:ri_max])))

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt3[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
colorbar=false,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="r",
yticks=([0,2.5,5,7.5,10],["0","2.5","5","7.5",""]),
)

#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
p61 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt2[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
xaxis="", 
xticks=([0,2.5,5,7.5,10],["","","","",""]),
yaxis="",
yticks=([0,2.5,5,7.5,10],["","","","",""]),
)

plot!([r[ri]], seriestype="hline", label="t=$(t)", linewidth=1.5, color = "white", legendfontsize=ls, style=:dash)
#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
plot!([r[ri]], seriestype="hline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)
#plot!([AH], seriestype="hline", label="", linewidth=1.5, color = clr_ah, legendfontsize=ls)
p62 = plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = clr_ah, legendfontsize=ls)

plot(r[ri_min:ri_max],  crlt2[ri_min:ri_max, ri], label="supercritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  crlt3[ri_min:ri_max, ri], label="subcritical", linewidth=2, style=:dash,
         xlim = (0,r[ri_max]), 
    #yticks=([0.02,0.04,0.06,0.08],["0.02","0.04","0.06","0.08"]),
    xticks=([0,2.5,5,7.5,10],["","","","",""])

)
plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = clr_ah, legendfontsize=ls)
p63 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

##########################################################

it = 69
crlt2, crlt3, t, AH = load_crtl23(it, crlt_i)
clim=( 
    min(minimum(crlt2[ri_min:ri_max,ri_min:ri_max]),minimum(crlt3[ri_min:ri_max,ri_min:ri_max])), 
    max(maximum(crlt2[ri_min:ri_max,ri_min:ri_max]),maximum(crlt3[ri_min:ri_max,ri_min:ri_max])))

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt3[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
colorbar=false,
legendfontsize=10,
xaxis="r", 
xticks=([0,2.5,5,7.5,10],["","2.5","5","7.5","10"]),
yaxis="r",
yticks=([0,2.5,5,7.5,10],["0","2.5","5","7.5",""]),
)

#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
p71 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        crlt2[ri_min:ri_max, ri_min:ri_max],
        color=clr,
        wsize = (900,700),
clim = clim,
ytickfont=10, yguidefontsize=10,
xtickfont=10, xguidefontsize=10,
legendfontsize=10,
xaxis="r", 
xticks=([0,2.5,5,7.5,10],["","2.5","5","7.5","10"]),
yaxis="",
yticks=([0,2.5,5,7.5,10],["","","","",""]),
)

plot!([r[ri]], seriestype="hline", label="t=$(t)", linewidth=1.5, color = "white", legendfontsize=ls, style=:dash)
#plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = clr_s, legendfontsize=ls)
plot!([r[ri]], seriestype="hline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)
#plot!([AH], seriestype="hline", label="", linewidth=1.5, color = clr_ah, legendfontsize=ls)
p72 = plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = clr_ah, legendfontsize=ls)

plot(r[ri_min:ri_max],  crlt2[ri_min:ri_max, ri], label="supercritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  crlt3[ri_min:ri_max, ri], label="subcritical", linewidth=2, style=:dash,
         xlim = (0,r[ri_max]), 
    #yticks=([0.02,0.04,0.06,0.08],["0.02","0.04","0.06","0.08"]),
    xticks=([0,2.5,5,7.5,10],["0","2.5","5","7.5","10"]),
    xlabel="r"
)
plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = clr_ah, legendfontsize=ls)
p73 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = clr_s, legendfontsize=ls, style=:dash)

##########################################################
plt = plot(
    p11, p12, p13,
    p21, p22, p23,
    p31, p32, p33,
    p41, p42, p43,
    p51, p52, p53,
    p61, p62, p63,
    p71, p72, p73,
    layout = grid(7, 3), size = (1000, 7*200))

savefig(plt, out_dir*"/crlt_$(crlt_i)_snapshots_ri$(ri)_slice.pdf")